In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import VGG16, MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from efficientnet import EfficientNetB3, center_crop_and_resize, preprocess_input

from keras import models
from keras import layers
from keras import callbacks

import time

import matplotlib.pyplot as plt

import cv2

Using TensorFlow backend.


In [3]:
BASEPATH = "stanford-dogs-dataset/"
BATCH_SIZE = 64
EPOCHS = 100

In [4]:
early_stopping = EarlyStopping(patience=20, verbose=1,restore_best_weights=True, monitor="val_acc")
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=7,verbose=1, monitor="val_acc")

In [5]:
def createModelInceptionResNetV2(trainable=False, optimizer="adam"):
    base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(120, activation="softmax"))

    if not trainable:
        for layer in base_model.layers:
            layer.trainable = False

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 
  
    return model

In [6]:
model = createModelInceptionResNetV2()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Using the ImageDataGenerator

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255.0,
                                   shear_range=0.1,
                                   rotation_range=10.,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=[0.9, 1.25],
                                   brightness_range=[0.5, 1.5],
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255.0)

validation_datagen = ImageDataGenerator(rescale=1./255.0)

In [8]:
train_generator = train_datagen.flow_from_directory(
    BASEPATH + "train",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    BASEPATH + "test",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    BASEPATH + "val",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 15394 images belonging to 120 classes.
Found 1114 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.


In [10]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = EPOCHS, callbacks=[early_stopping, reduce_lr])

Epoch 1/100
241/241 [==============================] - 254s 1s/step - loss: 1.5238 - acc: 0.6104 - val_loss: 0.8733 - val_acc: 0.8264
Epoch 2/100
241/241 [==============================] - 242s 1s/step - loss: 1.4113 - acc: 0.6383 - val_loss: 0.8456 - val_acc: 0.8310
Epoch 3/100
241/241 [==============================] - 242s 1s/step - loss: 1.3687 - acc: 0.6430 - val_loss: 0.8405 - val_acc: 0.8244
Epoch 4/100
241/241 [==============================] - 244s 1s/step - loss: 1.3135 - acc: 0.6468 - val_loss: 0.8692 - val_acc: 0.8269
Epoch 5/100
241/241 [==============================] - 242s 1s/step - loss: 1.2875 - acc: 0.6559 - val_loss: 0.8827 - val_acc: 0.8200
Epoch 6/100
241/241 [==============================] - 242s 1s/step - loss: 1.2329 - acc: 0.6690 - val_loss: 0.8302 - val_acc: 0.8183
Epoch 7/100
241/241 [==============================] - 243s 1s/step - loss: 1.2255 - acc: 0.6668 - val_loss: 0.8741 - val_acc: 0.8254
Epoch 8/100
241/241 [==============================] - 243s 1s

64/64 [==============================] - 27s 418ms/step - loss: 0.8248 - acc: 0.8318
Restoring model weights from the end of the best epoch.
241/241 [==============================] - 246s 1s/step - loss: 0.8977 - acc: 0.7417 - val_loss: 0.8248 - val_acc: 0.8318
Epoch 00037: early stopping


## Evaluation

In [11]:
loss, acc = model.evaluate_generator(test_generator,verbose=0, steps=test_generator.samples // BATCH_SIZE)

In [12]:
print(loss,acc)

0.8152329872051874 0.8330341


In [31]:
model.save("inceptionv3-model-{:.2f}acc-{:.2f}loss-{:.0f}.hdf5".format(acc, loss, time.time()))